# This notebook is all about star formation rate (SFR)

### How to plot it, how to make it smoother

### You want to use the last dataset so you have access to all the stars made throughout the sim

In [ ]:
# General script for SFR
# this can be run in just a jupyter notebook
# doesn't need to be submited as a job
# repeat for all sims you want to do this for

# imports
import yt
import numpy as np
import matplotlib.pyplot as plt
from yt.data_objects.particle_filters import add_particle_filter

# load in data
fiducial = yt.load("/mnt/research/galaxies-REU/sims/claire_isogal_sims/fiducial/DD0081/DD0081")

# make function for formed stars
def formed_star(pfilter, data):
    filter = data['all', 'creation_time'] >0
    return filter

# add the filter
add_particle_filter('formed_star', function=formed_star, filtered_type='all', requires=['creation_time'])
fiducial.add_particle_filter("formed_star")

# get all the data
ad = fiducial.all_data()
masses = ad["formed_star", "particle_mass"].in_units("Msun")
formation_time = ad["formed_star", "creation_time"].in_units("yr")

# make time range, bins, and histogram
time_range = [0, 4e9]  # 4 Gyr
n_bins = 1000
hist_fid, bins_fid = np.histogram(formation_time,bins=n_bins,range=time_range)

inds = np.digitize(formation_time, bins=bins_fid)
time = (bins_fid[:-1] + bins_fid[1:]) / 2

sfr = np.array([masses[inds == j + 1].sum() / (bins_fid[j + 1] - bins_fid[j]) for j in range(len(time))])
sfr[sfr == 0] = np.nan

# plot it
plt.plot(time / 1e6, sfr, label='Fiducial')
plt.yscale('log')
plt.xlabel("Time  [Myr]")
plt.ylabel(r"SFR  [M$_\odot$ yr$^{-1}$]")
plt.grid()
plt.legend();

## This section is for making a smoothed out version of SFR

### Made to run right after the above section

This can also just run in a jupyter notebook

In [ ]:
np.nan_to_num(sfr, True)

#function from https://stackoverflow.com/questions/14313510/how-to-calculate-rolling-moving-average-using-python-numpy-scipy
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

sfr_smooth = moving_average(sfr, 3)

new_time = np.linspace(1.5,4000,len(sfr_smooth))

plt.figure(figsize=(14,7))
plt.yscale('log')
plt.title('Smooth SFR Over Whole Sim')
plt.plot(new_time, sfr_smooth, label='Fiducial')
plt.xlabel('Time [Myr]')
plt.ylabel(r"SFR  [M$_\odot$ yr$^{-1}$]")
plt.grid()
plt.legend()

## Repeat these steps for all your sims, and then can play around with plotting them all together